***This notebook was adapted from**: (https://docs.qiime2.org/2022.11/tutorials/longitudinal/)

# Performing longitudinal and paired sample comparisons with `q2-longitudinal`

This tutorial will demonstrate the various features of `q2-longitudinal`, a plugin that supports statistical and visual comparisons of longitudinal study designs and paired samples, to determine if/how samples change between observational “states”. “States” will most commonly be related to time or an environmental gradient, and for paired analyses (`pairwise-distances` and `pairwise-differences`) the sample pairs should typically consist of the same individual subject observed at two different time points. For example, patients in a clinical study whose stool samples are collected before and after receiving treatment.

“States” can also commonly be methodological, in which case sample pairs will usually be the same individual at the same time with two different methods. For example, `q2-longitudinal` could compare the effects of different collection methods, storage methods, DNA extraction methods, or any bioinformatic processing steps on the feature composition of individual samples.

> Note

Many of the actions in q2-longitudinal take a metric value as input, which is usually a column name in a metadata file or a metadata-transformable artifact (including alpha diversity vectors, PCoA results, and many other QIIME 2 artifacts), or a feature ID in a feature table. The names of valid metric values in metadata files and metadata-transformable artifacts can be checked with the metadata tabulate command. Valid feature names (to use as metric values associated with a feature table) can be checked with the feature-data summarize command.

<div class="alert alert-block alert-info">
<p><b>Note</b></p>
<p>Many of the actions in <code>q2-longitudinal</code> take a <code>metric</code> value as input, which is usually a column name in a metadata file or a metadata-transformable artifact (including alpha diversity vectors, PCoA results, and many other QIIME 2 artifacts), or a feature ID in a feature table. The names of valid <code>metric</code> values in metadata files and metadata-transformable artifacts can be checked with the <a href="https://docs.qiime2.org/2022.11/tutorials/metadata/">metadata tabulate</a> command. Valid feature names (to use as <code>metric</code> values associated with a feature table) can be checked with the <code>feature-data summarize</code> command.</p>
</div>

The following flowchart illustrates the workflow involved in all <code>q2-longitudinal</code> analyses (<a href="https://docs.qiime2.org/2022.11/tutorials/overview/#key">figure key</a>). Each of these actions is described in more detail in the tutorials below.

<img src="https://docs.qiime2.org/2022.11/_images/longitudinal.png">

# STEP : Longitudinal Analysis

Using QIIME2 to create diversity analisys graphs and calculations.

- [QIIME2 Workflow Overview](https://docs.qiime2.org/2022.8/tutorials/overview/)


#### Methods
- [longitudinal](https://docs.qiime2.org/2022.11/plugins/available/longitudinal/#longitudinal)
- [longitudinal pairwise-differences](https://docs.qiime2.org/2022.11/plugins/available/longitudinal/pairwise-differences/)

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from shutil import rmtree
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.diversity.pipelines import alpha
from qiime2.plugins.taxa.methods import filter_table
from qiime2.plugins.taxa.methods import collapse

from qiime2.plugins.longitudinal.visualizers import pairwise_differences
from qiime2.plugins.feature_table.methods import relative_frequency


import matplotlib.pyplot as plt

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
metadata_file = '/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [3]:
# Parameters
experiment_name = "andressa-q20-trim_primer"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/data/manifest.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/data/metadata.tsv"
class_col = "state-name"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 12,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')
dada2_reps_path = os.path.join(qiime_folder, 'dada2-reps.qza')

alpha_path = os.path.join(qiime_folder, 'alpha-analysis')
beta_path = os.path.join(qiime_folder, 'beta-analysis')

# Iunput all SampleData[AlphaDiversity] Artifacts
alpha_files_paths = [os.path.join(qiime_folder, x) for x in os.listdir(alpha_path) if x.startswith('alpha-values')]

# Output -Longitudinal Artifacts

# Create folder to store longitudinal files
longitudinal_path = os.path.join(qiime_folder, 'longitudinal_analysis')
if not os.path.exists(longitudinal_path):
    os.makedirs(longitudinal_path)
    print(f'The new directory is created in {longitudinal_path}')

### Loading input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file, and all `SampleData[AlphaDiversity]` files.

In [6]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)
metadata_qa.to_dataframe()
print(f'Columns:\n{metadata_qa.columns}')

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)
tabs_df = tabs.view(Metadata).to_dataframe().T

# FeatureData[Sequence]
# reps = Artifact.load(dada2_reps_path)

# SampleData[AlphaDiversity]
alpha_artifacts = [Artifact.load(x) for x in alpha_files_paths]

In [52]:
# Get metrics names from inside each .qza files
metrics_names = list()

# Create a temporary folder to store extracted files
extract_path = os.path.join(base_dir, 'extracted')
if not os.path.exists(extract_path):
    os.makedirs(extract_path)
    print(f'The new directory is created in {extract_path}')

prefix_len = len('alpha-values-')
sufix_len = len('.qza')

# Extract each alpha diversity artifact and get the metric name
for art_path, art in zip(alpha_files_paths, alpha_artifacts):
    alpha_name = os.path.basename(art_path)[prefix_len:-sufix_len]
    out_path = os.path.join(extract_path, f'{alpha_name}')
    if not os.path.exists(out_path):
        os.makedirs(out_path)
        print(f'The new directory is created in {out_path}')
        art.extract(filepath=art_path, output_dir=out_path)
    extracted_dir = os.listdir(out_path)[0]
    tsv_path = os.path.join(out_path, extracted_dir, 'data', 'alpha-diversity.tsv')
    df = pd.read_csv(tsv_path, sep='\t')
    metric_name = df.columns[1]
    if metric_name != alpha_name:
        print(f'{metric_name} = {alpha_name}')
    metrics_names.append(metric_name)

# Clear temporary directories
# rmtree(extract_path)

The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/heip_e
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/michaelis_menten_fit
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/observed_features
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/pielou_e
pielou_evenness = pielou_e
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/brillouin_d
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/singles
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/robbins
The new directory is created in /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/extracted/simpson
The new directory is create

In [51]:
for alpha_file_path, metric in zip(alpha_files_paths, metrics_names):
    print(f'Processing Alpha file: {alpha_file_path} with metrics {metric}')
    
    # Define output visualization file path
    longitudinal_pairwise_diff_view = os.path.join(longitudinal_path, f'longitudinal-pairwise-differences-{metric}.qzv')
        
    !qiime longitudinal pairwise-differences \
      --m-metadata-file {metadata_file} \
      --m-metadata-file {alpha_file_path} \
      --p-metric {metric} \
      --p-group-column 'replic-id' \
      --p-state-column 'state-id' \
      --p-state-1 1 \
      --p-state-2 2 \
      --p-individual-id-column 'individual-id' \
      --p-replicate-handling random \
      --o-visualization {longitudinal_pairwise_diff_view}

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-heip_e.qza with metrics heip_e
Saved Visualization to: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/longitudinal_analysis/longitudinal-pairwise-differences-heip_e.qzv
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-michaelis_menten_fit.qza with metrics michaelis_menten_fit
^C

Aborted!
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-observed_features.qza with metrics observed_features
Saved Visualization to: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/longitudinal_analysis/longitudinal-pairwise-differences-observed_features.qzv
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-osd.qza with metrics osd
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature table to optionally use for 

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-dominance.qza with metrics dominance
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature table to optiona

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-ace.qza with metrics ace
^C

Aborted!
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-strong.qza with metrics strong
^C

Aborted!
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-gini_index.qza with metrics gini_index
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "me

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-berger_parker_d.qza with metrics berger_parker_d
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature tabl

^C
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-enspie.qza with metrics enspie
^C

Aborted!
Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-simpson_e.qza with metrics simpson_e
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-menhinick.qza with metrics menhinick
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature table to optiona

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-shannon.qza with metrics shannon_entropy
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature table to opt

Processing Alpha file: /home/lauro/nupeb/rede-micro/redemicro-andressa-lbtm/experiments/andressa-q20-trim_primer/qiime-artifacts/alpha-values-chao1.qza with metrics chao1
Usage: qiime longitudinal pairwise-differences [OPTIONS]

  Performs paired difference testing between samples from each subject.
  Sample pairs may represent a typical intervention study (e.g., samples
  collected pre- and post-treatment), paired samples from two different
  timepoints (e.g., in a longitudinal study design), or identical samples
  receiving different treatments. This action tests whether the change in a
  numeric metadata value "metric" differs from zero and differs between
  groups (e.g., groups of subjects receiving different treatments), and
  produces boxplots of paired difference distributions for each group. Note
  that "metric" can be derived from a feature table or metadata.

Inputs:
  --i-table ARTIFACT FeatureTable[RelativeFrequency]
                         Feature table to optionally use 